In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
import wandb
import os
import torch.optim as optimizers

In [3]:
import dfs_code
from torch_geometric.data import InMemoryDataset, Data
import pickle
import torch
import torch.nn as nn
import tqdm
import copy
import pandas as pd
#torch.multiprocessing.set_sharing_strategy('file_system') # this is important on local machine
#def set_worker_sharing_strategy(worker_id: int) -> None:
#    torch.multiprocessing.set_sharing_strategy('file_system')

In [4]:
import sys
sys.path = ['../../../src'] + sys.path
from dfs_transformer import EarlyStopping, DFSCodeSeq2SeqFC, smiles2graph, DFSCodeAutoencoder, DFSCodeSeq2SeqDecoder

2021-09-06 17:25:22.795670: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-06 17:25:22.795699: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
wandb.init(project='pubchem', entity='chrisxx')

config = wandb.config
config.n_atoms = 118
config.n_bonds = 5
config.emb_dim = 120
config.nhead = 12
config.nlayers = 3
config.n_memory = 2
config.max_nodes = 250
config.max_edges = 500
config.dim_feedforward = 2048
config.n_files = 64
config.n_splits = 16
config.n_iter_per_split = 1
config.lr = 0.0003
config.n_epochs = 10000
config.lr_adjustment_period = 500
config.patience = 5
config.factor = 0.96
config.minimal_lr = 6e-8
config.batch_size = 50
config.accumulate_grads = 4
config.valid_patience = 10000
config.valid_minimal_improvement=0.00
config.model_dir = "../../../models/pubchem/s2s/small/"
config.data_dir = "/mnt/project/pubchem_noH/"
config.pretrained_dir = None#"../../models/chembl/better_transformer/medium/"
config.num_workers = 0
config.prefetch_factor = 2
config.persistent_workers = False
config.load_last = True
config.gpu_id = 1


wandb: Currently logged in as: chrisxx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-09-06 17:25:35.160408: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-06 17:25:35.160438: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
os.makedirs(config.model_dir, exist_ok=True)

In [7]:
path = config.data_dir

In [8]:
class PubChem(Dataset):
    """PubChem dataset of molecules and minimal DFS codes."""
    def __init__(self, path=path, n_used = 8, n_splits = 64, max_nodes=config.max_nodes,
                 max_edges=config.max_edges, useHs=False, addLoops=False, memoryEfficient=False,
                 transform=None, n_mols_per_dataset=np.inf):
        self.path = path
        self.data = []
        self.path = path
        self.n_used = n_used
        self.n_splits = n_splits
        self.useHs = useHs
        self.addLoops = addLoops
        self.memoryEfficient = memoryEfficient
        self.n_mols_per_dataset = n_mols_per_dataset
        self.max_nodes = max_nodes
        self.max_edges = max_edges
        self.prepare()
        
        
    def prepare(self):
        codes_all = {}
        d_all = {}
        i2didx = {}
        perm = np.random.permutation(self.n_splits)
        for i in tqdm.tqdm(perm[:self.n_used]):
            dname = glob.glob(self.path+"/%d/min_dfs_codes_split*.json"%(i+1))[0]
            didx = int(dname.split("split")[-1][:-5])
            dname2 = self.path+"/%d/data_split%d.json"%(i+1, didx)
            with open(dname, 'r') as f:
                codes = json.load(f)
                for key, val in codes.items():
                    codes_all[key] = val
            with open(dname2, 'r') as f:
                d_dict = json.load(f)
                for key, val in d_dict.items():
                    d_all[key] = val
        
        for smiles, code in tqdm.tqdm(codes_all.items()):
            if len(self.data) > self.n_mols_per_dataset:
                break
            if code['min_dfs_code'] is not None and len(code['min_dfs_code']) > 1:
                d = d_all[smiles]
                #d = smiles2graph(smiles, useHs=self.useHs, addLoops=self.addLoops, skipCliqueCheck=False)
                if len(d['z']) > self.max_nodes:
                    continue
                if len(d['edge_attr']) > 2*self.max_edges:
                    continue
                
                data_ = Data(x=torch.tensor(d['x']),
                             z=torch.tensor(d['z']),
                             edge_attr=torch.tensor(d['edge_attr']),
                             edge_index=torch.tensor(d['edge_index'], dtype=torch.long),
                             min_dfs_code=torch.tensor(code['min_dfs_code']),
                             min_dfs_index=torch.tensor(code['dfs_index'], dtype=torch.long),
                             smiles=smiles)
                self.data += [data_]   
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [9]:
def collate_fn(dlist):
    x_batch = [] 
    z_batch = []
    edge_attr_batch = []
    rnd_code_batch = []
    min_code_batch = []
    for d in dlist:
        rnd_code, rnd_index = dfs_code.rnd_dfs_code_from_torch_geometric(d, 
                                                                         d.z.numpy().tolist(), 
                                                                         np.argmax(d.edge_attr.numpy(), axis=1))
        x_batch += [d.x]
        z_batch += [d.z]#[nn.functional.one_hot(d.z, 118)]#118 elements in periodic table
        edge_attr_batch += [d.edge_attr]
        rnd_code_batch += [torch.tensor(rnd_code)]
        min_code_batch += [d.min_dfs_code]
    return rnd_code_batch, x_batch, z_batch, edge_attr_batch, min_code_batch

In [10]:
ngpu=1
device = torch.device('cuda:%d'%config.gpu_id if (torch.cuda.is_available() and ngpu > 0) else 'cpu')

In [11]:
to_cuda = lambda T: [t.to(device) for t in T]

In [12]:
model = DFSCodeSeq2SeqDecoder(n_atoms=config.n_atoms,
                         n_bonds=config.n_bonds, 
                         emb_dim=config.emb_dim, 
                         nhead=config.nhead, 
                         nlayers=config.nlayers, 
                         max_nodes=config.max_nodes, 
                         max_edges=config.max_edges,
                         n_class_tokens=config.n_memory,
                         atom_encoder=nn.Embedding(config.n_atoms, config.emb_dim), 
                         bond_encoder=nn.Linear(config.n_bonds, config.emb_dim))

In [13]:
if config.pretrained_dir is not None:
    model.load_state_dict(torch.load(config.pretrained_dir+'checkpoint.pt', map_location=device))

In [14]:
if config.load_last:
    model.load_state_dict(torch.load(config.model_dir+'checkpoint.pt', map_location=device))

In [15]:
optim = optimizers.Adam(model.parameters(), lr=config.lr)

lr_scheduler = optimizers.lr_scheduler.ReduceLROnPlateau(optim, mode='min', verbose=True, patience=config.patience, factor=config.factor)
#lr_scheduler = optimizers.lr_scheduler.ExponentialLR(optim, gamma=config.factor)

early_stopping = EarlyStopping(patience=config.valid_patience, delta=config.valid_minimal_improvement,
                              path=config.model_dir+'checkpoint.pt')
bce = torch.nn.BCEWithLogitsLoss()
ce = torch.nn.CrossEntropyLoss(ignore_index=-1)
softmax = nn.Softmax(dim=2)

In [16]:
model.to(device)

DFSCodeSeq2SeqDecoder(
  (encoder): DFSCodeEncoder(
    (emb_dfs): PositionalEncoding(
      (dropout): Dropout(p=0, inplace=False)
    )
    (emb_seq): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (emb_atom): Embedding(118, 120)
    (emb_bond): Linear(in_features=5, out_features=120, bias=True)
    (mixer): Linear(in_features=600, out_features=600, bias=True)
    (enc): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=600, out_features=600, bias=True)
          )
          (linear1): Linear(in_features=600, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=600, bias=True)
          (norm1): LayerNorm((600,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((600,), eps=1e-05, elementwise_affine=True)
    

In [17]:
try:
    for epoch in range(config.n_epochs):
        epoch_loss = 0
        for split in range(config.n_splits):
            
            n_ids = config.n_files//config.n_splits
            dataset = PubChem(n_used = n_ids)
            loader = DataLoader(dataset, batch_size=config.batch_size, shuffle=True, pin_memory=False, collate_fn=collate_fn,
                   num_workers=config.num_workers, prefetch_factor=config.prefetch_factor, 
                    persistent_workers=config.persistent_workers)
            for j in range(config.n_iter_per_split):
                pbar = tqdm.tqdm(loader)
                for i, data in enumerate(pbar):
                    if i % config.accumulate_grads == 0: #bei 0 wollen wir das
                        optim.zero_grad()
                    rndc, x, z, eattr, minc = data
                    rndc = to_cuda(rndc)
                    z = to_cuda(z)
                    eattr = to_cuda(eattr)
                    minc = to_cuda(minc)
                    #prepare labels
                    minc_seq = nn.utils.rnn.pad_sequence(minc, padding_value=-1)

                    #prediction
                    dfs1, dfs2, atm1, atm2, bnd = model(rndc, z, eattr)
                    eos_label = (minc_seq[:,:,0] == (-1))
                    loss = ce(torch.reshape(dfs1, (-1, config.max_nodes)), minc_seq[:, :, 0].view(-1)) 
                    loss += ce(torch.reshape(dfs2, (-1, config.max_nodes)), minc_seq[:, :, 1].view(-1))
                    loss += ce(torch.reshape(atm1, (-1, config.n_atoms)), minc_seq[:, :, 2].view(-1))
                    loss += ce(torch.reshape(bnd, (-1, config.n_bonds)), minc_seq[:, :, 3].view(-1))
                    loss += ce(torch.reshape(atm2, (-1, config.n_atoms)), minc_seq[:, :, 4].view(-1))

                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
                    if (i+1) % config.accumulate_grads == 0:
                        optim.step() # bei 2 wollen wir das
                    epoch_loss = (epoch_loss*i + loss.item())/(i+1)

                    curr_lr = list(optim.param_groups)[0]['lr']
                    wandb.log({'loss':epoch_loss, 
                           'learning rate':curr_lr})
                    pbar.set_description('Epoch %d: CE %2.6f'%(epoch+1, epoch_loss))

                    if i % config.lr_adjustment_period == 0:
                        early_stopping(epoch_loss, model)
                        lr_scheduler.step(epoch_loss)

                
                if early_stopping.early_stop:
                    break

                if curr_lr < config.minimal_lr:
                    break
            del dataset
            del loader
        

except KeyboardInterrupt:
    torch.save(model.state_dict(), config.model_dir+'_keyboardinterrupt.pt')
    print('keyboard interrupt caught')


Epoch 1: CE 3.853438:   4%|█████▉                                                                                                                                              | 501/12494 [01:33<38:15,  5.22it/s]

EarlyStopping counter: 1 out of 10000


Epoch 1: CE 3.839766:   8%|███████████▊                                                                                                                                       | 1002/12494 [03:03<30:57,  6.19it/s]

EarlyStopping counter: 2 out of 10000


Epoch 1: CE 3.837922:  12%|█████████████████▋                                                                                                                                 | 1502/12494 [04:32<30:54,  5.93it/s]

...

EarlyStopping counter: 155 out of 10000


Epoch 2: CE 2.252438:  24%|███████████████████████████████████▎                                                                                                               | 3001/12494 [09:04<27:46,  5.70it/s]

EarlyStopping counter: 156 out of 10000
Epoch   632: reducing learning rate of group 0 to 8.6044e-06.


Epoch 2: CE 2.252285:  27%|███████████████████████████████████████▏                                                                                                           | 3327/12494 [10:03<27:42,  5.51it/s]


keyboard interrupt caught
